Reading the data

In [ ]:
require(readr)
trainSF <- read_csv("sanfrancisco_incidents_summer_2014.csv")
trainSEA <- read_csv("seattle_incidents_summer_2014.csv")


Converting the hours: When start and end points are given, a kind of mean is calculated

In [ ]:
require(lubridate)
trainSF$hour <- hour(hm(trainSF$Time))
trainSEA$hourS <- hour(mdy_hms(trainSEA$"Occurred Date or Date Range Start"))
trainSEA$hourE <- hour(mdy_hms(trainSEA$"Occurred Date Range End"))
trainSEA$hour <- floor((trainSEA$hourS + pmax(trainSEA$hourE,trainSEA$hourS,na.rm=TRUE)) / 2)

Counting by hour: dplyr is a functional approach to data handling upporting a pipeline syntax 
https://cran.rstudio.com/web/packages/dplyr/vignettes/introduction.html

In [ ]:
byhourSF <- trainSF %>% group_by(hour) %>% summarise(total = n())
byhourSEA <- trainSEA %>% group_by(hour) %>% summarise(total = n())

Plotting Seattle in blue, San Fransisco in red

In [ ]:
require(ggplot2)
p <- ggplot(byhourSF, aes(x = hour, y = total)) + geom_line(color="red")
p <- p + geom_line(data=byhourSEA,color="blue")
ggsave("hour.png", p, width=14, height=10, units="in")
p

Map plotting: 

Ideas taken from
https://www.kaggle.com/benhamner/sf-crime/san-francisco-top-crimes-map/files
